# Generatore Monte Carlo di raggi cosmici

La distribuzione angolare dei raggi cosmici che raggiungono il suolo terresrre segue approssimativamente la legge $\mathrm{d}P(\theta,\phi)/\mathrm{d}\Omega = k \cos^2\theta$ con $0\leq\theta<\pi/2$.

Il punto di impatto al suolo è uniformemente distribuito sulla superficie.

* Scrivere un generatore di raggi cosmici distribuiti secondo questo modello.
* Graficarne la distribuzione del punto di impatto e della direzione di provenienza.
* Graficare un numero finito con l'utility di matplotlib [quiver](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.quiver.html)

Opzionale:

* Definire la geometria di un semplice telescopio, ad es.: fatto da due piani quadrati orizzontali posti ad una certa altezza tra di loro
* Calcolare le intersezioni con i due piani di rivelatore dei raggi cosmici generati aventi punto di impatto sul piano di base
* Calcolare l'accettanza geometrica del telescopio rispetto ai raggi cosmici che arrivano al rivelatore di base